In [2]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://cli.github.com/packages stable InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,496 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,327 kB]
Fetched 8,952 kB in 1s (6,791 kB/s)
Reading 

In [3]:
!apt-get install poppler-utils libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libleptonica-dev is already the newest version (1.82.0-3build1).
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-script-latn is already the newest version (1:4.00~git30-7274cfa-1.1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.10).
python3-pil is already the newest version (9.0.1-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [4]:
!pip install unstructured[all-docs] unstructured-inference cmake python-dotenv pdf2image python-dateutil faiss-cpu sentence-transformers langchain==0.2.5 langchain-community==0.2.5 langchain-core==0.2.9 langchain-openai==0.1.9 bitsandbytes accelerate xformers triton transformers

In [5]:
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import elements_to_json

In [6]:
!pip install -U nltk

In [ ]:
# import shutil

# shutil.rmtree("bertv2")

In [13]:
import os
import pathlib

filename = "bertv2.pdf"

# 參數 max_characters, new_after_n_chars, combine_text_under_n_chars 只有在chunking_strategy不為None時才會生效 

elements_none = partition_pdf(
    filename,
    # chunking_strategy='by_title',
    # extract_image_block_types=["Image", "Table"],
    # infer_table_structure=False,
    # skip_infer_table_types=True,
    # max_characters=4000,
    # new_after_n_chars=3800,
    # combine_text_under_n_chars=2000,
    strategy='hi_res',
    # extract_image_block_output_dir=pathlib.Path(filename).stem
)

for idx, element in enumerate(elements_none):
  if element.category == "Title":
    print(idx, element)

print("**********************")
print(max([len(c.text) for c in elements_none]))

12 BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
13 Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova
16 Abstract
20 1 Introduction
28 2 Related Work
30 2.1 Unsupervised Feature-based Approaches
35 2.2 Unsupervised Fine-tuning Approaches
64 2.3 Transfer Learning from Supervised Data
67 3 BERT
77 3.1 Pre-training BERT
94 4 Experiments
95 3.2 Fine-tuning BERT
99 4.1 GLUE
115 4.2 SQuAD v1.1
129 4.3 SQuAD v2.0
137 4.4 SWAG
141 5 Ablation Studies
146 5.1 Effect of Pre-training Tasks
154 5.2 Effect of Model Size
159 5.3 Feature-based Approach with BERT
170 6 Conclusion
172 References
230 Appendix for “BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding”
238 A Additional Details for BERT
239 A.1 Illustration of the Pre-training Tasks
261 A.2 Pre-training Procedure
264 A.3 Fine-tuning Procedure
266 • Batch size: 16, 32
269 • Learning rate (Adam): 5e-5, 3e-5, 2e-5
270 • Number of epochs: 2, 3, 4
272 A.4 Comparison of BERT, 

In [20]:
partition_pdf?

## by_title chunking strategy

The by_title chunking strategy preserves section boundaries and optionally page boundaries as well. “Preserving” here means that a single chunk will never contain text that occurred in two different sections. When a new section starts, the existing chunk is closed and a new one started, even if the next element would fit in the prior chunk.
In addition to the behaviors of the basic strategy above, the by_title strategy has the following behaviors:

### Detect section headings.

A Title element is considered to start a new section. When a Title element is encountered, the prior chunk is closed and a new chunk started, even if the Title element would fit in the prior chunk.

### Respect page boundaries.

Page boundaries can optionally also be respected using the multipage_sections argument. This defaults to True meaning that a page break does not start a new chunk. Setting this to False will separate elements that occur on different pages into distinct chunks.

### Combine small sections.

In certain documents, partitioning may identify a list-item or other short paragraph as a Title element even though it does not serve as a section heading. This can produce chunks substantially smaller than desired. This behavior can be mitigated using the combine_text_under_n_chars argument. This defaults to the same value as max_characters such that sequential small sections are combined to maximally fill the chunking window. Setting this to 0 will disable section combining.

### max_characters

This is a hard cap on the maximum number of characters in a single chunk.

It ensures that no chunk will exceed this limit, regardless of sentence or paragraph boundaries.

Think of it as the "upper bound" for chunk size.

👉 Example: if max_characters=500, no chunk will ever contain more than 500 characters.

### new_after_n_chars

This is a soft chunking guide that tries to create a new chunk once the text length hits this threshold.

Unlike max_characters, it allows the library to "look for a natural break" (e.g., end of a sentence or paragraph) around that point.

It’s useful if you want chunks to be around a certain size but still respect natural text boundaries.

👉 Example: if new_after_n_chars=300, the library will attempt to start a new chunk after ~300 characters, but it might go a bit over if the next break point is at 320 characters.

In [15]:
elements_by_title = partition_pdf(
    filename,
    chunking_strategy='by_title',
    # extract_image_block_types=["Image", "Table"],          # optional
    # infer_table_structure=False,
    skip_infer_table_types=True,
    max_characters=4000,
    new_after_n_chars=3800,
    # combine_text_under_n_chars=2000,
    strategy='hi_res',
    # extract_image_block_output_dir=pathlib.Path(filename).stem
)

print(max([len(c.text) for c in elements_by_title]))

3930


In [24]:
print(min([len(c.text) for c in elements_by_title]))

655


這一段特別短，看一下原因

In [26]:
for idx, c in enumerate(elements_by_title):
  if len(c.text) == 655:
    print(idx, c.text)

18 A.5 Illustrations of Fine-tuning on Different Tasks

The illustration of ﬁne-tuning BERT on different tasks can be seen in Figure 4. Our task-speciﬁc models are formed by incorporating BERT with one additional output layer, so a minimal num- ber of parameters need to be learned from scratch. Among the tasks, (a) and (b) are sequence-level tasks while (c) and (d) are token-level tasks. In the ﬁgure, E represents the input embedding, Ti represents the contextual representation of token i, [CLS] is the special symbol for classiﬁcation out- put, and [SEP] is the special symbol to separate non-consecutive token sequences.

B Detailed Experimental Setup


上一段的長度

In [27]:
len(elements_by_title[17].text)

3930

下一段的長度

In [28]:
len(elements_by_title[19].text)

3437

### Title 作為 "Chunk" 的起始點

但也許不是完全通用的，因為可能有些段落過長，超過max_characters，以至於被強制拆開並開啟下一個Chunk.

### by_title 策略

主要原則：每個標題 (title element) 會被視為一個「新 Chunk」的起始點。

也就是說，文件裡的結構（例如章節標題、節標題）會決定 Chunk 的邊界。

標題底下的內容（段落、表格、清單等）會被歸類到同一個 Chunk。

#### 如果段落過長，超過了 max_characters，系統會強制把它拆分成多個 Chunk。

- 這是為了避免生成特別龐大的 Chunk。

- 所以標題雖然是起始點，但不是唯一的切割依據。

#### 如果有設定 new_after_n_chars，則在達到這個字數附近時，系統會「嘗試」找個自然的分割點切開。

- 例如在句子結束處換 Chunk，而不是硬斷在句子中間。

- 這會讓 Chunk 看起來更自然，但仍然受 max_characters 上限約束。

In [16]:
print(elements_by_title[1].text)

1 Introduction

Language model pre-training has been shown to be effective for improving many natural language processing tasks (Dai and Le, 2015; Peters et al., 2018a; Radford et al., 2018; Howard and Ruder, 2018). These include sentence-level tasks such as natural language inference (Bowman et al., 2015; Williams et al., 2018) and paraphrasing (Dolan and Brockett, 2005), which aim to predict the re- lationships between sentences by analyzing them holistically, as well as token-level tasks such as named entity recognition and question answering, where models are required to produce ﬁne-grained output at the token level (Tjong Kim Sang and De Meulder, 2003; Rajpurkar et al., 2016).

We argue that current techniques restrict the power of the pre-trained representations, espe- cially for the ﬁne-tuning approaches. The ma- jor limitation is that standard language models are unidirectional, and this limits the choice of archi- tectures that can be used during pre-training. For example, in 

In [17]:
print(elements_by_title[2].text)

2.1 Unsupervised Feature-based Approaches

Learning widely applicable representations of words has been an active area of research for decades, including non-neural (Brown et al., 1992; Ando and Zhang, 2005; Blitzer et al., 2006) and neural (Mikolov et al., 2013; Pennington et al., 2014) methods. Pre-trained word embeddings are an integral part of modern NLP systems, of- fering signiﬁcant improvements over embeddings learned from scratch (Turian et al., 2010). To pre- train word embedding vectors, left-to-right lan- guage modeling objectives have been used (Mnih and Hinton, 2009), as well as objectives to dis- criminate correct from incorrect words in left and right context (Mikolov et al., 2013).

These approaches have been generalized to coarser granularities, such as sentence embed- dings (Kiros et al., 2015; Logeswaran and Lee, 2018) or paragraph embeddings (Le and Mikolov, 2014). To train sentence representations, prior work has used objectives to rank candidate next sentences (Je

In [18]:
print(elements_by_title[3].text)

2.3 Transfer Learning from Supervised Data

There has also been work showing effective trans- fer from supervised tasks with large datasets, such as natural language inference (Conneau et al., 2017) and machine translation (McCann et al., 2017). Computer vision research has also demon- strated the importance of transfer learning from large pre-trained models, where an effective recipe is to ﬁne-tune models pre-trained with Ima- geNet (Deng et al., 2009; Yosinski et al., 2014).

Model Architecture BERT’s model architec- ture is a multi-layer bidirectional Transformer en- coder based on the original implementation de- scribed in Vaswani et al. (2017) and released in the tensor2tensor library.1 Because the use of Transformers has become common and our im- plementation is almost identical to the original, we will omit an exhaustive background descrip- tion of the model architecture and refer readers to Vaswani et al. (2017) as well as excellent guides such as “The Annotated Transformer.”2


In [19]:
print(elements_by_title[4].text)

3 BERT

We introduce BERT and its detailed implementa- tion in this section. There are two steps in our framework: pre-training and ﬁne-tuning. Dur- ing pre-training, the model is trained on unlabeled data over different pre-training tasks. For ﬁne- tuning, the BERT model is ﬁrst initialized with the pre-trained parameters, and all of the param- eters are ﬁne-tuned using labeled data from the downstream tasks. Each downstream task has sep- arate ﬁne-tuned models, even though they are ini- tialized with the same pre-trained parameters. The question-answering example in Figure 1 will serve as a running example for this section.

A distinctive feature of BERT is its uniﬁed ar- chitecture across different tasks. There is mini-

In this work, we denote the number of layers (i.e., Transformer blocks) as L, the hidden size as H, and the number of self-attention heads as A.3 We primarily report results on two model sizes: BERTBASE (L=12, H=768, A=12, Total Param- eters=110M) and BERTLARGE (L=2

In [22]:
elements_by_title_2 = partition_pdf(
    filename,
    chunking_strategy='by_title',
    # extract_image_block_types=["Image", "Table"],
    # infer_table_structure=False,
    form_extraction_skip_tables=True,
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=500,
    strategy='hi_res',
    # extract_image_block_output_dir=pathlib.Path(filename).stem
)

print(max([len(c.text) for c in elements_by_title_2]))

3888


In [23]:
print(min([len(c.text) for c in elements_by_title_2]))

503


In [5]:
import pathlib

filename = 'bertv2.pdf'

elements = partition_pdf(
    filename,
    chunking_strategy='by_title',
    extract_image_block_types=["Image", "Table"],
    # infer_table_structure=False,
    form_extraction_skip_tables=True,
    max_characters=4000,
    new_after_n_chars=3800,
    strategy='hi_res',
    extract_image_block_output_dir=pathlib.Path(filename).stem
)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

圖像處理

In [4]:
pip install pdf2image pillow layoutparser opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 56.8 MB/s eta 0:00:00
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=f6f922b4113a58950df86db71281f17fad541d99306844ab29d1ac2bd03d17f6
  Stored in directory: /root/.cache/pip/wheels/7c/96/04/4f5f31ff812f684f69f40cb1634357812220aac58d4698048c
Successfully built iopath


In [11]:
import layoutparser as lp
from pdf2image import convert_from_path
import cv2
import numpy as np

# 1. 將 PDF 轉成圖片 (這裡取第一頁)
pdf_path = "bertv2.pdf"
pages = convert_from_path(pdf_path, dpi=150)
pages[14].save("example.png", "PNG")


In [12]:
from unstructured.partition.image import partition_image

# Returns a List[Element] present in the pages of the parsed image document
elements_image = partition_image("example.png", strategy='hi_res', extract_image_block_types=["Image"])

In [22]:
elements_image[8].text

'Figure 4: Illustrations of Fine-tuning BERT on Different Tasks.'

In [23]:
elements_image[8]

單就圖像提取來說，比partition_pdf看起來很好多。所以，依照你的需求或是數據來源，你可能會需要訂製化自己的數據提取流程


## PDF -> Vectorstore

你現在有了文字，表格，圖像。你可以開始建立自己專屬的數據庫

### 文字:
- 原始數據
- 進行壓縮 (像是summary)

### 表格/圖像:
- 使用image caption將圖片轉換成文字

In [7]:
import os
import configparser

from langchain_openai import ChatOpenAI


def credential_init():

  credential_file = "credentials.ini"

  if os.path.exists(credential_file):
      credentials = configparser.ConfigParser()
      credentials.read(credential_file)
      os.environ['OPENAI_API_KEY'] = credentials['openai'].get('api_key')
  else:
      os.environ['OPENAI_API_KEY'] = os.environ['OPENAI']

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
           model_name="gpt-4o-mini", temperature=0)

In [11]:
import io
import base64

from PIL import Image
from langchain_core.prompts.image import ImagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.docstore.document import Document
from langchain_core.runnables import chain, RunnablePassthrough, RunnableLambda


def build_standard_chat_prompt_template(kwargs):
  messages = []

  if 'system' in kwargs:
    content = kwargs.get('system')

    # allow list of prompts for multimodal
    if isinstance(content, list):
      prompts = [PromptTemplate(**c) for c in content]
    else:
      prompts = [PromptTemplate(**content)]

    message = SystemMessagePromptTemplate(prompt=prompts)
    messages.append(message)

  if 'human' in kwargs:
    content = kwargs.get('human')

    # allow list of prompts for multimodal
    if isinstance(content, list):
      prompts = []
      for c in content:
        if c.get("type") == "image":
          prompts.append(ImagePromptTemplate(**c))
        else:
          prompts.append(PromptTemplate(**c))
    else:
      if content.get("type") == "image":
        prompts = [ImagePromptTemplate(**content)]
      else:
        prompts = [PromptTemplate(**content)]

    message = HumanMessagePromptTemplate(prompt=prompts)
    messages.append(message)

  chat_prompt_template = ChatPromptTemplate.from_messages(messages)

  return chat_prompt_template

### 文字壓縮

In [8]:
prompt = f"Summarize the following text:\n\n{elements[0]}\n\nSummary:"

In [9]:
prompt

'Summarize the following text:\n\n9\n\n2019\n\n1\n\n0\n\n2\n\ny a M 4 2 ] L C . s c [ 2 v 5 0 8 4 0 . 0 1 8 1 :\n\nv\n\narXiv\n\ni\n\nX\n\nr\n\na\n\nBERT: Pre-training of Deep Bidirectional Transformers for Language Understanding\n\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\n\nGoogle AI Language\n\n{jacobdevlin,mingweichang,kentonl,kristout}@google.com\n\nAbstract\n\nWe introduce a new language representa- tion model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language repre- sentation models (Peters et al., 2018a; Rad- ford et al., 2018), BERT is designed to pre- train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a re- sult, the pre-trained BERT model can be ﬁne- tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substan

In [10]:
model.invoke(prompt)

AIMessage(content='The text introduces BERT (Bidirectional Encoder Representations from Transformers), a new language representation model developed by Google AI Language. BERT pre-trains deep bidirectional representations from unlabeled text by conditioning on both left and right context in all layers. This allows the pre-trained BERT model to be fine-tuned with just one additional output layer for various tasks like question answering and language inference, achieving state-of-the-art results without significant task-specific modifications. BERT outperforms previous models, achieving new benchmarks on eleven natural language processing tasks, including significant improvements in GLUE score, MultiNLI accuracy, and SQuAD question answering tests.', response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 549, 'total_tokens': 679, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0

In [11]:
text_prompt_template = {"template": "Summarize the following text:\n\n{query}\n\nSummary:", "input_variables": ["query"]}

input_ = {
    "human": [text_prompt_template],
}

chat_prompt_template = build_standard_chat_prompt_template(input_)

text_pipeline = chat_prompt_template|model|StrOutputParser()

batches = [{"query": c.text} for c in elements]

results = await text_pipeline.abatch(batches)

In [17]:
documents = []

for result in results:

  documents.append(Document(page_content=result, metadata={'filename': "bertv2.pdf",
                               'type': "text"}))

In [18]:
documents[-1]

Document(page_content='The text discusses the impact of the number of training steps and different masking strategies on the performance of BERT models. \n\n**Effect of Number of Training Steps:**\n1. **Pre-training Duration:** BERTBASE achieves about 1.0% higher accuracy on the MNLI dataset when pre-trained for 1 million steps compared to 500,000 steps, indicating the necessity of extensive pre-training.\n2. **MLM vs. LTR Pre-training:** The Masked Language Model (MLM) converges slightly slower than the Left-to-Right (LTR) model but starts outperforming the LTR model in terms of absolute accuracy almost immediately.\n\n**Ablation for Different Masking Procedures:**\n- **Masking Strategies:** BERT uses a mixed strategy (80% MASK, 10% SAME, 10% RND) to reduce the mismatch between pre-training and fine-tuning stages.\n- **Evaluation:** The study evaluates the effect of different masking strategies on MNLI and NER datasets. Fine-tuning is robust to different masking strategies, but the fe

願意的話，你甚至可以辦到標註是在哪一頁

### 表格和影像提取

In [17]:
import aiofiles
import asyncio
from operator import itemgetter
from pathlib import Path


# Async function to read image and convert to base64
@chain
async def image_to_base64(image_path: str) -> str:
  loop = asyncio.get_event_loop()

  print(image_path)

  # Use threadpool to avoid blocking
  def read_image():
    with Image.open(image_path) as image:
      buffered = io.BytesIO()
      image.convert("RGB").save(buffered, format="JPEG")
      return base64.b64encode(buffered.getvalue()).decode('utf-8')

  return await loop.run_in_executor(None, read_image)


text_prompt_template = {"template": "Describe the content."}
image_prompt_template = {"type": "image",
              "template": {"url": "data:image/jpeg;base64,{image_str}"},
              "input_variables": ["image_str"]}

input_ = {
    "human": [text_prompt_template, image_prompt_template],
}

chat_prompt_template = build_standard_chat_prompt_template(input_)

# Async chain for converting image_path -> base64
# The lambda in RunnablePassthrough.assign is async-friendly\

image_2_image_str_chain = RunnablePassthrough.assign(image_str=lambda x: itemgetter('image_path')|image_to_base64)


image_caption_pipeline = image_2_image_str_chain | chat_prompt_template| model | StrOutputParser()

batches = [{"image_path": os.path.join("bertv2", c)} for c in os.listdir("bertv2") if 'table' in Path(c).stem]

# results = await image_caption_pipeline.abatch(batches)

In [30]:
for result in results:

  documents.append(Document(page_content=result, metadata={'filename': "bertv2.pdf",
                               "type": "table"}))

In [31]:
documents[-1]

Document(page_content='The content is a table comparing the performance of different systems on a specific task, likely related to natural language processing or machine learning, as indicated by the metrics used (EM and F1 scores). The table is divided into three sections: "Top Leaderboard Systems," "Published," and "Ours."\n\n1. **Top Leaderboard Systems (Dec 10th, 2018):**\n   - **Human:** \n     - Dev EM: 86.3\n     - Dev F1: 89.0\n     - Test EM: 86.9\n     - Test F1: 89.5\n   - **#1 Single - MIR-MRC (F-Net):**\n     - Test EM: 74.8\n     - Test F1: 78.0\n   - **#2 Single - nlnet:**\n     - Test EM: 74.2\n     - Test F1: 77.1\n\n2. **Published:**\n   - **unet (Ensemble):**\n     - Test EM: 71.4\n     - Test F1: 74.9\n   - **SLQA+ (Single):**\n     - Test EM: 71.4\n     - Test F1: 74.4\n\n3. **Ours:**\n   - **BERT_LARGE (Single):**\n     - Dev EM: 78.7\n     - Dev F1: 81.9\n     - Test EM: 80.0\n     - Test F1: 83.1\n\nThe table shows that the BERT_LARGE model outperforms the other

基於Documents內容建立vectorstore

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


NameError: name 'documents' is not defined

### Save the vectorstore

In [33]:
vectorstore.save_local("vectorstore")

### Load the vectorstore

In [35]:
vectorstore_1 = FAISS.load_local(
    "vectorstore", embeddings, allow_dangerous_deserialization=True
)

vectorstore_1.docstore._dict

{'60e01e6d-6674-40b3-b10e-c68aa6047d5a': Document(page_content='BERT (Bidirectional Encoder Representations from Transformers) is a new language representation model introduced by Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova from Google AI Language. Unlike previous models, BERT pre-trains deep bidirectional representations by conditioning on both left and right context in all layers. This allows BERT to be fine-tuned with just one additional output layer for various tasks like question answering and language inference, achieving state-of-the-art results without significant task-specific modifications. BERT outperforms existing models on eleven natural language processing tasks, including substantial improvements in GLUE score, MultiNLI accuracy, and SQuAD question answering benchmarks.', metadata={'filename': 'bertv2.pdf', 'type': 'text'}),
 '7628d053-6431-44e6-854f-d63f272cd9eb': Document(page_content='The introduction discusses the effectiveness of language model 

### 合併vectorstore。


In [ ]:
vectorstore_1.merge_from(vectorstore_2)

In [40]:
os.makedirs("codex")

In [12]:
async def pdf_2_vectorstore(filename, embeddings):

  image_dir = Path(filename).stem

  elements = partition_pdf(
    filename,
    chunking_strategy='by_title',
    extract_image_block_types=["Table"],
    # infer_table_structure=False,
    form_extraction_skip_tables=True,
    max_characters=4000,
    new_after_n_chars=3800,
    strategy='hi_res',
    extract_image_block_output_dir=image_dir
)

  documents = []

  text_prompt_template = {"template": "Summarize the following text:\n\n{query}\n\nSummary:", "input_variables": ["query"]}

  input_ = {
      "human": [text_prompt_template],
  }

  chat_prompt_template = build_standard_chat_prompt_template(input_)

  text_pipeline = chat_prompt_template|model|StrOutputParser()

  batches = [{"query": c.text} for c in elements]

  results = await text_pipeline.abatch(batches)

  for result in results:

    documents.append(Document(page_content=result, metadata={'filename': image_dir,
                                 'type': "text"}))


  text_prompt_template = {"template": "Describe the content."}
  image_prompt_template = {"type": "image",
                "template": {"url": "data:image/jpeg;base64,{image_str}"},
                "input_variables": ["image_str"]}

  input_ = {
      "human": [text_prompt_template, image_prompt_template],
  }

  chat_prompt_template = build_standard_chat_prompt_template(input_)

  # Async chain for converting image_path -> base64
  # The lambda in RunnablePassthrough.assign is async-friendly\

  image_2_image_str_chain = RunnablePassthrough.assign(image_str=lambda x: itemgetter('image_path')|image_to_base64)


  image_caption_pipeline = image_2_image_str_chain | chat_prompt_template| model | StrOutputParser()

  batches = [{"image_path": os.path.join(image_dir, c)} for c in os.listdir(image_dir) if 'table' in Path(c).stem]

  results = await image_caption_pipeline.abatch(batches)

  for result in results:

    documents.append(Document(page_content=result, metadata={'filename': image_dir,
                                "type": "table"}))

  vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

  return vectorstore

In [18]:
from operator import itemgetter
from pathlib import Path
from tqdm import tqdm

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

vectorstore_list = []

for filename in tqdm(os.listdir("codex")):

  print(f"\n****\n{filename}\n****\n")

  vectorstore = await pdf_2_vectorstore(os.path.join("codex", filename), embeddings)

  vectorstore_list.append(vectorstore)

  0%|          | 0/3 [00:00<?, ?it/s]


****
Codex - Aeldari.pdf
****

Codex - Aeldari/table-21-8.jpg
Codex - Aeldari/table-9-2.jpg
Codex - Aeldari/table-11-3.jpg
Codex - Aeldari/table-79-36.jpg
Codex - Aeldari/table-47-21.jpg
Codex - Aeldari/table-71-33.jpg
Codex - Aeldari/table-61-28.jpg
Codex - Aeldari/table-121-54.jpg
Codex - Aeldari/table-35-15.jpg
Codex - Aeldari/table-105-48.jpg
Codex - Aeldari/table-65-30.jpg
Codex - Aeldari/table-27-11.jpg
Codex - Aeldari/table-91-41.jpg
Codex - Aeldari/table-67-31.jpg
Codex - Aeldari/table-49-22.jpg
Codex - Aeldari/table-7-1.jpg
Codex - Aeldari/table-51-23.jpg
Codex - Aeldari/table-63-29.jpg
Codex - Aeldari/table-31-13.jpg
Codex - Aeldari/table-69-32.jpg
Codex - Aeldari/table-53-24.jpg
Codex - Aeldari/table-55-25.jpg
Codex - Aeldari/table-25-10.jpg
Codex - Aeldari/table-115-51.jpg
Codex - Aeldari/table-45-20.jpg
Codex - Aeldari/table-103-47.jpg
Codex - Aeldari/table-93-42.jpg
Codex - Aeldari/table-37-16.jpg
Codex - Aeldari/table-81-37.jpg
Codex - Aeldari/table-123-55.jpg
Codex - A

 33%|███▎      | 1/3 [26:37<53:14, 1597.19s/it]


****
Codex - Adeptus Mechanicus.pdf
****

Codex - Adeptus Mechanicus/table-107-22.jpg
Codex - Adeptus Mechanicus/table-97-16.jpg
Codex - Adeptus Mechanicus/table-107-21.jpg
Codex - Adeptus Mechanicus/table-101-19.jpg
Codex - Adeptus Mechanicus/table-88-9.jpg
Codex - Adeptus Mechanicus/table-89-10.jpg
Codex - Adeptus Mechanicus/table-94-14.jpg
Codex - Adeptus Mechanicus/table-119-23.jpg
Codex - Adeptus Mechanicus/table-120-26.jpg
Codex - Adeptus Mechanicus/table-120-28.jpg
Codex - Adeptus Mechanicus/table-120-29.jpg
Codex - Adeptus Mechanicus/table-120-27.jpg
Codex - Adeptus Mechanicus/table-96-15.jpg
Codex - Adeptus Mechanicus/table-92-13.jpg
Codex - Adeptus Mechanicus/table-119-25.jpg
Codex - Adeptus Mechanicus/table-119-24.jpg
Codex - Adeptus Mechanicus/table-83-6.jpg
Codex - Adeptus Mechanicus/table-64-5.jpg
Codex - Adeptus Mechanicus/table-99-17.jpg
Codex - Adeptus Mechanicus/table-91-12.jpg
Codex - Adeptus Mechanicus/table-3-1.jpg
Codex - Adeptus Mechanicus/table-89-11.jpg
Codex 

 67%|██████▋   | 2/3 [50:33<25:02, 1502.27s/it]


****
Codex -Black Templars.pdf
****

Codex -Black Templars/table-9-2.jpg
Codex -Black Templars/table-15-8.jpg
Codex -Black Templars/table-27-14.jpg
Codex -Black Templars/table-9-3.jpg
Codex -Black Templars/table-7-1.jpg
Codex -Black Templars/table-31-16.jpg
Codex -Black Templars/table-35-18.jpg
Codex -Black Templars/table-13-5.jpg
Codex -Black Templars/table-15-7.jpg
Codex -Black Templars/table-17-9.jpg
Codex -Black Templars/table-25-13.jpg
Codex -Black Templars/table-29-15.jpg
Codex -Black Templars/table-13-6.jpg
Codex -Black Templars/table-19-10.jpg
Codex -Black Templars/table-21-11.jpg
Codex -Black Templars/table-23-12.jpg
Codex -Black Templars/table-11-4.jpg
Codex -Black Templars/table-33-17.jpg


100%|██████████| 3/3 [1:00:31<00:00, 1210.36s/it]


In [19]:
merged_store = vectorstore_list[0]
for vs in vectorstore_list[1:]:
  merged_store.merge_from(vs)

In [22]:
merged_store.save_local("warhammer40k_codex")